In [38]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
import logging

logging.basicConfig(level=logging.DEBUG)

COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [43]:
safety_trigger = False

In [44]:
PROJECT_NAME = 'person'
ROOT_DIR = '/home/mosminin/datasets/COCO2017'



coco_dataset_dir = os.path.join(ROOT_DIR, 'coco2017_all_classes')
dataset_name = f'coco_{PROJECT_NAME}'

# Настройки экспорта
TARGET_CLASSES = ['person'] # Список нужных классов
yolov4_export_dir = os.path.join(ROOT_DIR, f'coco2017_yolov4_{PROJECT_NAME}') # Целевая папка
label_field = "ground_truth"

# ПАРСИНГ ДАТАСЕТА COCO-2017 ДЛЯ YOLOV4

## СКАЧИВАЕМ ПОЛНЫЙ ДАТАСЕТ COCO-2017

In [45]:
if not safety_trigger:
    # Выполнение кода грохнет текущий датасет и начнёт долгое скачивание заново!
    dataset = foz.load_zoo_dataset(
        "coco-2017",
        splits=["train", "validation", "test"],
        dataset_dir=coco_dataset_dir,
        dataset_name=dataset_name,
        dataset_type=fo.types.COCODetectionDataset,
        classes=COCO_CLASSES,
        drop_existing_dataset=True,
        # only_matching=False,
        shuffle=True,
        # max_samples=1000
    )
    safety_trigger = True
else:
    print('Check safety trigger!')

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/home/mosminin/datasets/COCO2017/coco2017_all_classes/train' if necessary


Found annotations at '/home/mosminin/datasets/COCO2017/coco2017_all_classes/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/home/mosminin/datasets/COCO2017/coco2017_all_classes/raw/instances_train2017.json'


Sufficient images already downloaded


INFO:fiftyone.utils.coco:Sufficient images already downloaded


...

Dataset is unlabeled; ignoring classes requirement


 100% |█████████████| 40670/40670 [6.3s elapsed, 0s remaining, 6.7K samples/s]       


INFO:eta.core.utils: 100% |█████████████| 40670/40670 [6.3s elapsed, 0s remaining, 6.7K samples/s]       


Dataset 'coco_person' created


INFO:fiftyone.zoo.datasets:Dataset 'coco_person' created


## ПООЧЕРЁДНО ЗАГРУЖАЕМ КАЖДЫЙ СПЛИТ И ЭКСПОРТИРУЕМ В СООТВЕТСТВУЮЩУЮ ПАПКУ

In [46]:
dataset = fo.load_dataset(dataset_name)

In [47]:
import shutil
from tqdm.notebook import tqdm
from pathlib import Path
from time import sleep

splits = ['train', 'validation', 'test'] # Splits for export
paths = {}

for split in splits:
    split_view = dataset.match_tags(split)
    if split == 'validation':
        split = 'valid'  # For Yolo compatibility
    export_dir = Path(os.path.join(yolov4_export_dir, split))
    logging.info(f'{export_dir=}')
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv4Dataset,
        label_field=label_field,
        classes=TARGET_CLASSES,
    )

    yolov4_export_dir = Path(yolov4_export_dir)
    split_path = Path(os.path.join(yolov4_export_dir, split))
    data_path = Path(os.path.join(yolov4_export_dir, split,'data'))  # Abs path
    logging.debug(f'{split_path=}')
    logging.debug(f'{data_path=}')

    for file_path in split_path.glob('*'):
        if file_path.is_dir():
            continue
        file_path.unlink()
        logging.info(f'{file_path} is deleted')

    # Читаем все .jpg файлы в /data
    logging.info(f'Reading all files in data_path to file_list')
    file_list = []
    for file_path in tqdm(list(data_path.glob('*.jpg'))):
        file_list.append(str(file_path.resolve()))



    # Записываем список файлов в {split}.txt
    logging.info(f'Writing file_list to annotations file')
    annotations_file = yolov4_export_dir / f'{split}.txt'
    with open(annotations_file, 'w') as f:
        for file_path in tqdm(file_list):
            # print(f'{file_path=}')
            f.write(file_path+'\n')

    # Добавляем пути к сплитам в словарь
    paths[split] = str(annotations_file)




num_classes = len(TARGET_CLASSES)
classes_file_name = yolov4_export_dir / 'obj.names'
data_file_name = yolov4_export_dir / 'obj.data'
backup_dir_name = yolov4_export_dir / 'backup'

# Создать папку backup
backup_dir_name.mkdir(parents=True, exist_ok=True)
paths['backup'] = str(backup_dir_name)

# Сформировать obj.names и obj.data

with open(classes_file_name, "w") as f:
    for name in TARGET_CLASSES:
        f.write(name)

with open(data_file_name, "w") as f:
    # Добавить в obj.data кол-во классов, пути на аннотации и бэкап
    f.write(f'classes = {num_classes}\n')
    for key, value in paths.items():
        f.write(f'{key} = {value}\n')
    f.write(f'names = {classes_file_name}\n')




INFO:root:export_dir=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train')


 100% |███████████| 117266/117266 [3.5m elapsed, 0s remaining, 605.1 samples/s]       


INFO:eta.core.utils: 100% |███████████| 117266/117266 [3.5m elapsed, 0s remaining, 605.1 samples/s]       
DEBUG:root:split_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train')
DEBUG:root:data_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data')
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/obj.names is deleted
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/images.txt is deleted
INFO:root:Reading all files in data_path to file_list


  0%|          | 0/117266 [00:00<?, ?it/s]

INFO:root:Writing file_list to annotations file


...

 100% |█████████████| 40670/40670 [33.9s elapsed, 0s remaining, 1.8K samples/s]       


INFO:eta.core.utils: 100% |█████████████| 40670/40670 [33.9s elapsed, 0s remaining, 1.8K samples/s]       
DEBUG:root:split_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test')
DEBUG:root:data_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test/data')
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test/obj.names is deleted
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test/images.txt is deleted
INFO:root:Reading all files in data_path to file_list


  0%|          | 0/40670 [00:00<?, ?it/s]

INFO:root:Writing file_list to annotations file


  0%|          | 0/40670 [00:00<?, ?it/s]

## СКРИПТ ДЛЯ РАСПРЕДЕЛЕНИЯ ДАТАСЕТА

- dataset   - valid - .jpg / .txt
            - train - .jpg / .txt
            - test  - .jpg

    - Для каждой папки перебрать .jpg и составить список train/valid/test.txt
    - Файлы со списками сохранить в dataset
    - В dataset папку backup
    - В dataset создать файл obj.data:
        classes =   {кол-во_классов}
        train =     {абсолютный пусть до train.txt}
        valid =     ...
        names =     {абсолютный пусть до obj.names}
        backup =    {абсолютный пусть до папки backup}

# БЭКАПИЛКА

In [ ]:
import os
import time
import tarfile

def make_archive(dir_path):
    now = time.strftime('%Y_%m_%d_%H%M%S')
    archive_name = f'{dir_path}_{now}.tar'
    i = 0
    while os.path.exists(archive_name):
        i += 1
        archive_name = f'{dir_path}_{now}_{i}.tar'
    with tarfile.open(archive_name, 'w') as archive:
        archive.add(dir_path, arcname=os.path.basename(dir_path), mode='a')